In [ ]:
'''
# 20 Questions style game vs GPT
# pip install openai

from dotenv import load_dotenv 
from openai import OpenAI
import random
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
# ====== 1. 설정 ======
client = OpenAI(api_key=OPENAI_API_KEY)

WORDS = ["사과", "자동차", "고양이", "컴퓨터", "영화", "바다", "책", "커피", "로봇", "피자"]
SECRET_WORD = random.choice(WORDS)

print("🤖 20고개 게임 시작!")
print("제가 단어 하나를 골랐어요. 질문을 통해 맞춰보세요.")
print("사람과 AI가 번갈아 가며 질문과 답변을 주고받습니다.\n")

turn = 0
history = []  # 대화 기록 저장

# ====== 2. 메인 루프 ======
while True:
    turn += 1
    print(f"\n--- 턴 {turn} ---")

    # 사람 차례
    human_q = input("🙋 당신의 질문: ")
    if human_q.strip() == "":
        continue

    # GPT가 사람의 질문에 답변 (정답은 SECRET_WORD)
    ai_answer = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"너는 20고개 게임의 진행자야. 정답은 '{SECRET_WORD}'이야."},
            {"role": "user", "content": f"질문: {human_q} — 정답 '{SECRET_WORD}' 기준으로 예/아니오/간단한 설명으로 대답해."}
        ]
    ).choices[0].message.content

    print(f"🤖 GPT의 답변: {ai_answer}")

    # 사람이 추측할 기회
    guess = input("💡 정답을 추측하시겠습니까? (없으면 Enter): ")
    if guess.strip() == SECRET_WORD:
        print("🎉 당신이 정답을 맞혔습니다! 인간 승리!")
        break

    # ===== GPT 차례 =====
    prompt = f"""
지금까지의 Q&A 기록:
{history}

정답은 '{SECRET_WORD}'입니다.
너는 이번 턴에서 사람에게 질문을 하나 해야 합니다.
질문은 예/아니오로 답할 수 있도록 간단하게 만들어야 합니다.
"""
    ai_question = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "너는 20고개 게임 참가자야."},
            {"role": "user", "content": prompt}
        ]
    ).choices[0].message.content.strip()

    print(f"\n🧠 GPT의 질문: {ai_question}")
    human_answer = input("🙋 당신의 답변 (예/아니오/모름): ")

    # GPT가 추측 시도
    guess_prompt = f"""
지금까지의 Q&A 기록:
{history}
가장 최근 질문: {ai_question}
사람의 답변: {human_answer}

정답 후보를 하나 추측해봐.
한 단어만 출력해.
"""
    ai_guess = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "너는 20고개 게임 참가자야."},
            {"role": "user", "content": guess_prompt}
        ]
    ).choices[0].message.content.strip()

    print(f"🤖 GPT의 추측: {ai_guess}")

    if ai_guess == SECRET_WORD:
        print("🏆 GPT가 정답을 맞혔습니다! AI 승리!")
        break

    # 기록 업데이트
    history.append({
        "human_q": human_q,
        "ai_answer": ai_answer,
        "ai_q": ai_question,
        "human_answer": human_answer,
        "ai_guess": ai_guess
    })
'''

In [1]:

# 다른 형태의 코드 

# 20 Questions Game (LLM chooses the secret word)
# -----------------------------------------------
# pip install openai

from openai import OpenAI

# OpenAI 클라이언트 설정
from dotenv import load_dotenv 
import os
load_dotenv()
# client = OpenAI(api_key="YOUR_OPENAI_API_KEY")

print("🎯 20고개 게임 시작!")
print("LLM이 마음속으로 어떤 단어를 떠올렸습니다.")
print("당신은 질문을 던져 그 단어를 맞혀야 합니다.\n")

# ===== 1. LLM이 비밀 단어 선택 =====
choose_word_prompt = """
너는 20고개 게임의 진행자야.
사람에게 알려주지 말고 머릿속으로 단어 하나를 떠올려.
일상적인 명사 하나만 골라 (예: 사과, 고양이, 비행기 등).
그 단어는 숨기고, '단어를 정했습니다!'라고만 출력해.
"""
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": "너는 게임 마스터야."},
              {"role": "user", "content": choose_word_prompt}]
)
print("🤖 GPT:", response.choices[0].message.content.strip())

# ===== 2. 게임 루프 =====
history = []
turn = 0

while True:
    turn += 1
    print(f"\n--- 턴 {turn} ---")

    # 플레이어 질문 입력
    question = input("🙋 질문: ")
    if not question.strip():
        continue

    # GPT가 답변 (단어는 비밀로 유지)
    answer_prompt = f"""
너는 20고개 게임의 진행자야.
너는 이미 마음속으로 단어 하나를 골랐어.
그 단어는 공개하지 마.

사람의 질문: {question}

정답 단어 기준으로 '예', '아니오', 또는 간단한 설명으로만 대답해.
"""
    answer = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "너는 게임 진행자야."},
            {"role": "user", "content": answer_prompt}
        ]
    ).choices[0].message.content.strip()

    print(f"🤖 GPT의 답변: {answer}")

    # 플레이어가 추측할 기회
    guess = input("💡 정답을 추측하시겠습니까? (없으면 Enter): ").strip()
    if guess:
        check_prompt = f"""
너는 20고개 게임의 진행자야.
너는 이미 마음속으로 단어 하나를 골랐어.
그 단어는 공개하지 말고, 사용자가 입력한 단어가 정답인지 판단해.

사용자 추측: {guess}
정답이면 '정답입니다!'라고, 아니면 '틀렸습니다!'라고만 답하세요.
"""
        check = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "너는 게임 진행자야."},
                {"role": "user", "content": check_prompt}
            ]
        ).choices[0].message.content.strip()

        print(f"🤖 GPT: {check}")

        if "정답" in check:
            print("🎉 축하합니다! 당신이 단어를 맞혔습니다!")
            break

    # 20턴 제한 (20고개)
    if turn >= 20:
        print("\n😅 20번의 질문이 끝났습니다! 정답은 공개되지 않습니다 (GPT만 알고 있음).")
        break



🎯 20고개 게임 시작!
LLM이 마음속으로 어떤 단어를 떠올렸습니다.
당신은 질문을 던져 그 단어를 맞혀야 합니다.



NameError: name 'client' is not defined